# Order Lifecycle

Lifecycle analytics using `wolfpack/order_events.csv`.

This tracks:
- Submitted, partially filled, filled, canceled counts
- Fill ratio and time-to-final-status
- Final status by execution tier

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

from QuantConnect import *
from QuantConnect.Research import QuantBook

qb = QuantBook()
print('QuantBook initialized')


def read_csv_from_store(key):
    try:
        if not qb.ObjectStore.ContainsKey(key):
            print(f'ObjectStore key not found: {key}')
            return None
        content = qb.ObjectStore.Read(key)
        if not content:
            print(f'Empty ObjectStore key: {key}')
            return None
        return pd.read_csv(StringIO(content))
    except Exception as e:
        print(f'Error reading {key}: {e}')
        return None


## Data Loading — Order Events Log

Loads the order events log from ObjectStore, parses tier and week-ID tags from the order annotation string, and displays the first few rows to confirm the schema. Each row represents one status update (submitted, partially filled, filled, cancelled) for a single order. The `tier` column inferred from tags is the key dimension used to segment all lifecycle statistics that follow.

## Order-Level Summary — Fill Ratio and Days to Final

This cell pivots from status-event rows to one row per order, computing fill ratio, days-to-final-status, and final outcome for each order ID. The resulting `order_summary` DataFrame is the primary analysis unit — one order with its full lifecycle condensed into a single record. The head preview confirms that `fill_ratio` and `days_to_final` are populated before charts are generated.

## Order Lifecycle — 4-Panel Chart

These four panels summarize the order lifecycle from submission to final resolution. Top-left shows counts by final status (filled, cancelled, etc.); top-right shows the fill-ratio distribution to reveal how often orders are only partially filled; bottom-left shows the final status mix as a stacked proportion by tier, testing whether high-tier orders fill more reliably; and bottom-right shows time-to-final-status by tier, checking whether strong-tier market-price limits resolve faster than wide weak-tier limits.

## Cancel Rate and Fill Ratio Scorecard by Tier

This summary table reports cancel rate and average fill ratio for each signal tier, combining lifecycle performance into a compact scorecard. A higher cancel rate for weak-tier orders is expected given their 1.5% limit offset, but if strong-tier orders also show elevated cancels it may indicate the stale-limit cancellation threshold is too aggressive. Compare average fill ratios across tiers to confirm the tiered limit-offset design achieves meaningfully different fill outcomes.

In [ ]:
import re

df_events = read_csv_from_store('wolfpack/order_events.csv')
if df_events is None:
    raise ValueError('order_events.csv is required. Run a backtest with order-event logging enabled.')

df_events['date'] = pd.to_datetime(df_events['date'])
for col in ['quantity', 'fill_quantity', 'fill_price', 'limit_price']:
    if col in df_events.columns:
        df_events[col] = pd.to_numeric(df_events[col], errors='coerce').fillna(0.0)

def parse_tag_value(tag, key):
    if pd.isna(tag):
        return np.nan
    m = re.search(rf'{key}=([^;]+)', str(tag))
    return m.group(1) if m else np.nan

df_events['tier'] = df_events['tag'].apply(lambda t: parse_tag_value(t, 'tier')).fillna('unknown')
df_events['week_id'] = df_events['tag'].apply(lambda t: parse_tag_value(t, 'week_id')).fillna('')

print(f'order events: {len(df_events):,}')
display(df_events.head())


In [ ]:
# Aggregate each order lifecycle
grp = df_events.sort_values('date').groupby('order_id', as_index=False)

order_summary = grp.agg(
    symbol=('symbol', 'first'),
    tier=('tier', 'first'),
    order_type=('order_type', 'first'),
    quantity=('quantity', 'first'),
    submitted_at=('date', 'min'),
    final_at=('date', 'max')
)

final_status = (
    df_events.sort_values('date')
             .groupby('order_id')
             .tail(1)[['order_id', 'status']]
             .rename(columns={'status': 'final_status'})
)

fills = (
    df_events.groupby('order_id', as_index=False)['fill_quantity']
             .sum()
             .rename(columns={'fill_quantity': 'filled_qty'})
)

order_summary = order_summary.merge(final_status, on='order_id', how='left')
order_summary = order_summary.merge(fills, on='order_id', how='left')

order_summary['abs_qty'] = order_summary['quantity'].abs().replace(0, np.nan)
order_summary['fill_ratio'] = (order_summary['filled_qty'].abs() / order_summary['abs_qty']).fillna(0.0).clip(0, 1)
order_summary['days_to_final'] = (order_summary['final_at'] - order_summary['submitted_at']).dt.days.fillna(0)

display(order_summary.head())


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

status_counts = order_summary['final_status'].value_counts().sort_values(ascending=False)
status_counts.plot(kind='bar', ax=axes[0, 0], color='#1f77b4')
axes[0, 0].set_title('Final Order Status Counts')
axes[0, 0].set_ylabel('Count')
axes[0, 0].grid(axis='y', alpha=0.3)

sns.histplot(order_summary['fill_ratio'], bins=20, ax=axes[0, 1], color='#2ca02c')
axes[0, 1].set_title('Fill Ratio Distribution')
axes[0, 1].set_xlabel('Fill ratio')
axes[0, 1].grid(alpha=0.3)

tier_status = pd.crosstab(order_summary['tier'], order_summary['final_status'], normalize='index')
tier_status = tier_status.reindex(['strong', 'moderate', 'weak', 'exit', 'unknown']).dropna(how='all')
tier_status.plot(kind='bar', stacked=True, ax=axes[1, 0], colormap='tab20')
axes[1, 0].set_title('Final Status Mix by Tier')
axes[1, 0].set_ylabel('Share')
axes[1, 0].grid(axis='y', alpha=0.3)

sns.boxplot(data=order_summary, x='tier', y='days_to_final', order=['strong', 'moderate', 'weak', 'exit', 'unknown'], ax=axes[1, 1])
axes[1, 1].set_title('Days to Final Status by Tier')
axes[1, 1].set_xlabel('Tier')
axes[1, 1].set_ylabel('Days')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
cancel_rate = (
    order_summary.assign(is_canceled=order_summary['final_status'].astype(str).str.contains('Canceled', case=False, na=False))
                .groupby('tier', as_index=False)
                .agg(orders=('order_id', 'count'),
                     cancel_rate=('is_canceled', 'mean'),
                     avg_fill_ratio=('fill_ratio', 'mean'))
                .sort_values('orders', ascending=False)
)
display(cancel_rate)
